## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-18-16-40-15 +0000,nyt,Demonstrators Gather Across the U.S. for ‘No K...,https://www.nytimes.com/2025/10/18/us/protests...
1,2025-10-18-16-38-40 +0000,nypost,Mets owner Steve Cohen donates over $1M to NYC...,https://nypost.com/2025/10/18/us-news/ny-mets-...
2,2025-10-18-16-33-31 +0000,cbc,Day of nationwide 'No Kings' protests against ...,https://www.cbc.ca/news/world/no-kings-protest...
3,2025-10-18-16-28-12 +0000,nyt,"For Prince Andrew, a Steady Fall From Grace En...",https://www.nytimes.com/2025/10/18/world/europ...
4,2025-10-18-16-27-36 +0000,nyt,"Chen Ning Yang, Nobel-Winning Physicist, Is De...",https://www.nytimes.com/2025/10/18/science/che...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2351/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,52
101,ukraine,14
188,new,13
21,prince,11
22,andrew,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
131,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...,109
152,2025-10-18-00-06-00 +0000,wsj,President Trump has commuted the sentence of f...,https://www.wsj.com/politics/trump-commutes-se...,102
187,2025-10-17-22-26-18 +0000,nypost,Trump orders lying ex-Long Island Rep. George ...,https://nypost.com/2025/10/17/us-news/trump-or...,101
225,2025-10-17-20-49-26 +0000,nypost,"Trump urges Russia, Ukraine to end war ‘where ...",https://nypost.com/2025/10/17/us-news/trump-sa...,100
10,2025-10-18-16-05-14 +0000,startribune,Live: Thousands expected at anti-Trump ‘No Kin...,https://www.startribune.com/live-thousands-exp...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
131,109,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...
263,70,2025-10-17-18-19-43 +0000,nypost,Jeffrey Epstein-linked Prince Andrew forced to...,https://nypost.com/2025/10/17/world-news/princ...
225,48,2025-10-17-20-49-26 +0000,nypost,"Trump urges Russia, Ukraine to end war ‘where ...",https://nypost.com/2025/10/17/us-news/trump-sa...
136,47,2025-10-18-02-00-54 +0000,latimes,Dozens of 'No Kings' protests will be held ac...,https://www.latimes.com/california/story/2025-...
18,44,2025-10-18-15-51-37 +0000,nypost,"Vice President Vance, Witkoff headed to Israel...",https://nypost.com/2025/10/18/world-news/vice-...
157,40,2025-10-17-23-50-52 +0000,nypost,Salesforce CEO Marc Benioff apologizes for say...,https://nypost.com/2025/10/17/us-news/salesfor...
24,36,2025-10-18-15-26-23 +0000,nypost,NYC teen shooters and victims skyrocket since ...,https://nypost.com/2025/10/18/us-news/teen-sho...
142,34,2025-10-18-01-24-15 +0000,nypost,Leather-wearing Democratic state lawmaker Scot...,https://nypost.com/2025/10/17/us-news/leather-...
241,32,2025-10-17-19-56-00 +0000,wsj,Bank of Canada Gov. Tiff Macklem said data nex...,https://www.wsj.com/articles/bank-of-canada-su...
166,31,2025-10-17-23-37-23 +0000,wapo,U.S. military detains survivors of latest boat...,https://www.washingtonpost.com/national-securi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
